In [18]:
!pip install tensorflow
!pip install numpy
!pip install matplotlib
!pip install pillow
!pip install keras

In [51]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Directory paths
input_dir = 'input_images'
target_dir = 'target_images'
intermediate_dir = 'intermediate_images'

In [61]:
# Function to load images from a directory
def load_images(image_dir):
    images = []
    for filename in sorted(os.listdir(image_dir)):
        if filename.endswith('.png'):
            img_path = os.path.join(image_dir, filename)
            img = load_img(img_path, target_size=(512, 512))
            img = img_to_array(img)
            img = (img / 127.5) - 1  # Normalize to [-1, 1]
            images.append(img)
    return np.array(images)

# Load input and target images
input_images = load_images(input_dir)
target_images = load_images(target_dir)

# Load intermediate images
intermediate_images = []
for subdir in sorted(os.listdir(intermediate_dir)):
    intermediate_subdir = os.path.join(intermediate_dir, subdir)
    if os.path.isdir(intermediate_subdir):
        images = load_images(intermediate_subdir)
        intermediate_images.append(images)

# Create TensorFlow datasets
input_dataset = tf.data.Dataset.from_tensor_slices(input_images)
target_dataset = tf.data.Dataset.from_tensor_slices(target_images)

# Convert intermediate images list to tuple of datasets
intermediate_datasets = tuple(tf.data.Dataset.from_tensor_slices(steps) for steps in intermediate_images)

# Combine into one dataset
dataset = tf.data.Dataset.zip((input_dataset, intermediate_datasets, target_dataset))

# Shuffle and batch the dataset
batch_size = 32
dataset = dataset.shuffle(buffer_size=1000).batch(batch_size)

# Split into training and validation sets
total_samples = len(input_images) + sum(len(steps) for steps in intermediate_images)
train_size = int(0.8 * total_samples)

# Split into training and validation sets
val_size = total_samples - train_size
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size).take(val_size)

# Print dataset information
# Print dataset information
print("Number of training samples:", train_size)
print("Number of validation samples:", val_size)

Number of training samples: 12
Number of validation samples: 3


In [62]:
# Print lengths of input, intermediate, and target images
print("Number of input images:", len(input_images))
print("Number of intermediate images folders:", len(intermediate_images))
print("Number of target images:", len(target_images))

# Print number of images in each intermediate folder
for i, steps in enumerate(intermediate_images):
    print(f"Number of images in intermediate folder {i+1}:", len(steps))


Number of input images: 3
Number of intermediate images folders: 3
Number of target images: 3
Number of images in intermediate folder 1: 4
Number of images in intermediate folder 2: 4
Number of images in intermediate folder 3: 4


In [63]:
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model

# Function to build the generator (U-Net architecture)
def build_generator(input_shape):
    inputs = Input(shape=input_shape)
    
    # Encoder
    conv1 = Conv2D(64, 4, strides=2, padding='same', activation='relu')(inputs)
    conv2 = Conv2D(128, 4, strides=2, padding='same', activation='relu')(conv1)
    conv3 = Conv2D(256, 4, strides=2, padding='same', activation='relu')(conv2)
    conv4 = Conv2D(512, 4, strides=2, padding='same', activation='relu')(conv3)
    
    # Decoder
    deconv1 = Conv2DTranspose(256, 4, strides=2, padding='same', activation='relu')(conv4)
    deconv1_concat = Concatenate()([deconv1, conv3])
    deconv2 = Conv2DTranspose(128, 4, strides=2, padding='same', activation='relu')(deconv1_concat)
    deconv2_concat = Concatenate()([deconv2, conv2])
    deconv3 = Conv2DTranspose(64, 4, strides=2, padding='same', activation='relu')(deconv2_concat)
    deconv3_concat = Concatenate()([deconv3, conv1])
    outputs = Conv2DTranspose(3, 4, strides=2, padding='same', activation='tanh')(deconv3_concat)
    
    return Model(inputs, outputs)

# Function to build the discriminator (PatchGAN architecture)
def build_discriminator(input_shape):
    inputs = Input(shape=input_shape)
    
    conv1 = Conv2D(64, 4, strides=2, padding='same', activation='relu')(inputs)
    conv2 = Conv2D(128, 4, strides=2, padding='same', activation='relu')(conv1)
    conv3 = Conv2D(256, 4, strides=2, padding='same', activation='relu')(conv2)
    conv4 = Conv2D(512, 4, strides=2, padding='same', activation='relu')(conv3)
    outputs = Conv2D(1, 4, strides=1, padding='same', activation='sigmoid')(conv4)
    
    return Model(inputs, outputs)

# Define input shape
input_shape = (512, 512, 3)

# Build generator and discriminator
generator = build_generator(input_shape)
discriminator = build_discriminator(input_shape)

# Display model summaries
generator.summary()
discriminator.summary()



Model: "functional_31"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_16      │ (None, 512, 512,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_58 (Conv2D)  │ (None, 256, 256,  │      3,136 │ input_layer_16[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_59 (Conv2D)  │ (None, 128, 128,  │    131,200 │ conv2d_58[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_60 (Conv2D)  │ (None, 64, 64,    │    524,544 │ conv2d_59[0][0]   │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_61 (Conv2D)  │ (None, 32, 32,    │  2,097,664 │ conv2d_60[0][0]   │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_24 │ (None, 64, 64,    │  2,097,408 │ conv2d_61[0][0]   │
│ (Conv2DTranspose)   │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_18      │ (None, 64, 64,    │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 512)              │            │ conv2d_60[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_25 │ (None, 128, 128,  │  1,048,704 │ concatenate_18[0… │
│ (Conv2DTranspose)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_19      │ (None, 128, 128,  │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 256)              │            │ conv2d_59[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_26 │ (None, 256, 256,  │    262,208 │ concatenate_19[0… │
│ (Conv2DTranspose)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_20      │ (None, 256, 256,  │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 128)              │            │ conv2d_58[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_27 │ (None, 512, 512,  │      6,147 │ concatenate_20[0… │
│ (Conv2DTranspose)   │ 3)                │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 6,171,011 (23.54 MB)

 Trainable params: 6,171,011 (23.54 MB)

 Non-trainable params: 0 (0.00 B)

Model: "functional_33"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_17 (InputLayer)     │ (None, 512, 512, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_62 (Conv2D)              │ (None, 256, 256, 64)   │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_63 (Conv2D)              │ (None, 128, 128, 128)  │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_64 (Conv2D)              │ (None, 64, 64, 256)    │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_65 (Conv2D)              │ (None, 32, 32, 512)    │     2,097,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_66 (Conv2D)              │ (None, 32, 32, 1)      │         8,193 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,764,737 (10.55 MB)

 Trainable params: 2,764,737 (10.55 MB)

 Non-trainable params: 0 (0.00 B)

In [72]:
import keras
from keras import layers
from keras.optimizers import Adam

# Define optimizer for both generator and discriminator
optimizer = Adam(learning_rate=0.0002, beta_1=0.5)

# Compile discriminator
discriminator.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'], run_eagerly=True)

# Compile generator
generator.compile(optimizer=optimizer, loss='binary_crossentropy', run_eagerly=True)

# Define GAN model
discriminator.trainable = False  # Set discriminator to non-trainable
gan_input = Input(shape=input_shape)
generated_image = generator(gan_input)
gan_output = discriminator(generated_image)
gan = Model(gan_input, gan_output)

# Compile GAN
gan.compile(optimizer=optimizer, loss='binary_crossentropy', run_eagerly=True)


In [104]:
import numpy as np

# Define number of epochs and batch size
epochs = 2
batch_size = 1

# # Define function to generate batches of real images
# def generate_real_samples(dataset, batch_size):
#     idx = np.random.randint(0, dataset.shape[0], batch_size)
#     real_images = dataset[idx]
#     labels = np.ones((batch_size, 1))  # Label real images as 1 (real)
#     return real_images, labels

# Define function to generate batches of real images
def generate_real_samples(dataset, batch_size):
    # Shuffle and batch the dataset
    # dataset = dataset.shuffle(buffer_size=10).batch(batch_size)
    # Get a batch of images and labels
    iterator = iter(dataset)
    
    real_images = next(iterator)
    
    # Convert real_images to TensorFlow tensor if it's a tuple
    if isinstance(real_images, tuple):
        print("conversionz")
        real_images = tf.convert_to_tensor(real_images)
    
    # print(type(real_images))
    
    # Ensure the shape of real_images is as expected
    expected_shape = (batch_size, 512, 512, 3)  # Define the expected shape
    if real_images.shape != expected_shape:
        print("conversion")
        real_images = tf.reshape(real_images, expected_shape)
    
    labels = tf.ones((batch_size, 1))  # Label real images as 1 (real)
    return real_images, labels

# Define function to train the discriminator

def train_discriminator(discriminator, generator, dataset, batch_size):
    # Generate real samples
    real_images, real_labels = generate_real_samples(dataset, batch_size)

    # Remove the extra dimension from real_images
    # real_images = real_images[1:]
    modified_tup = tf.reshape(real_images, (3, 512, 512, 3))
    print(type(real_images))
    print(real_images)

    # Generate fake samples
    noise = np.random.randn(batch_size, 512, 512, 3)
    
    # Remove the extra dimension from fake_images
    fake_images = generator.predict(noise)#[1:]
    print(type(fake_images))

    fake_labels = np.zeros((batch_size, 1))  # Label fake images as 0 (fake)

    # Train discriminator on real and fake samples
    d_loss_real = discriminator.train_on_batch(real_images, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    
    print(d_loss)
    
    return d_loss


# Define function to train the generator
def train_generator(gan_model, discriminator, batch_size):
    # noise = np.random.randn(batch_size, latent_dim)
    noise = np.random.randn(batch_size, 512, 512, 3)
    labels = np.ones((batch_size, 1))  # Label generated images as 1 (real)
    g_loss = gan_model.train_on_batch(noise, labels)
    
    print(g_loss)
    
    return g_loss

# Main training loop
for epoch in range(epochs):
    print(len(dataset))
    print(batch_size)
    print(len(dataset) // batch_size)
    for batch in range(len(dataset) // batch_size):
        # Train discriminator
        d_loss = train_discriminator(discriminator, generator, dataset, batch_size)

        # Train generator (via GAN model)
        g_loss = train_generator(gan, discriminator, batch_size)

        # Print loss after each batch
        print(f'Epoch {epoch + 1}/{epochs} - Batch {batch + 1}/{len(dataset) // batch_size} - Discriminator Loss: {d_loss[0]}, Generator Loss: {g_loss}')

    # Optionally, monitor training progress after each epoch
    # print(f'Epoch {epoch + 1}/{epochs} - Discriminator Loss: {d_loss[0]}, Generator Loss: {g_loss}')


1
1
1
conversionz


2024-05-19 03:40:36.575725: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Shapes of all inputs must match: values[0].shape = [3,512,512,3] != values[1].shape = [3,3,512,512,3]


InvalidArgumentError: {{function_node __wrapped__Pack_N_3_device_/job:localhost/replica:0/task:0/device:CPU:0}} Shapes of all inputs must match: values[0].shape = [3,512,512,3] != values[1].shape = [3,3,512,512,3] [Op:Pack] name: 